# Mod 2 Project  - ask Aroa for that SQL code!


## Introduction

This project is an exploration into football team results in Europe in 2011, and how rain affected the performaces in those matches.

We will first have a look at the information in an SQL database off Kaggle (the open source data website for data science practice. 

Then we will the Pandas library to manipulate the data, and access weather data for using an API (DarkSky). 

Once we've put all the information together, we will export the summary statistics into a Mongo Database.

The statistics we want are as follows:

    The name of the team
    The total number of goals scored by the team during the 2011 season
    The total number of wins the team earned during the 2011 season
    A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
    The team's win percentage on days where it was raining during games in the 2011 season.

## Importing the relevant libraries 

In [64]:
import sqlite3
import pandas as pd
import numpy as np
import requests
import pymongo
from pymongo import MongoClient
import json
import os
import matplotlib.pyplot as plt
%matplotlib inline
import lib 

## Importing the data and having a first look

In [2]:
conn = sqlite3.connect('database.sqlite') ##accessing the saved database in this github repo
cur = conn.cursor()

cur.execute("""SELECT * FROM MATCHES
                WHERE (Season = '2011') AND NOT(Div = 'E0')""") ##selecting the needed data 

In [3]:
df1 = pd.DataFrame(cur.fetchall()) ##putting the data into a pandas Dataframe

df1.columns = [x[0] for x in cur.description] ## relabeling the columns correctly

df_original = df1.copy() ## making a copy for future reference 

In [4]:
df1.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [5]:
df1.isna().sum()  ##checking for null values in our columns

Match_ID    0
Div         0
Season      0
Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
dtype: int64

We've got a nice looking clean dataframe! Time to get to work on it...

# Calculating some stats 

## Total Goals Scored

Will calculate by splitting up teams at home and away, then combining the two results

In [6]:
df1_home = df1.drop(['Match_ID','Div','Season','Date','AwayTeam','FTAG','FTR'], axis=1)

In [7]:
df1_home = df1_home.groupby(['HomeTeam'], sort=True).sum()
#drop irrelevant columns, then find total goals for teams at home

In [8]:
df1_home = df1_home.reset_index()

In [9]:
df1_home.head()

,HomeTeam,FTHG
0,Aachen,15
1,Augsburg,20
2,Bayern Munich,49
3,Bochum,23
4,Braunschweig,21


In [10]:
df1_away = df1.drop(['Match_ID','Div','Season','Date','HomeTeam','FTHG','FTR'], axis=1)

In [11]:
df1_away = df1_away.groupby(['AwayTeam'], sort=True).sum()
#drop irrelevant columns, then find total goals for teams away from home

In [12]:
df1_away = df1_away.reset_index()

In [13]:
df1_away.head()

,AwayTeam,FTAG
0,Aachen,15
1,Augsburg,16
2,Bayern Munich,28
3,Bochum,18
4,Braunschweig,16


In [14]:
df1_final = pd.concat([df1_home,df1_away],axis=1,sort=True)
#combine two dataframes into a total running dataframe

In [15]:
df1_final = df1_final.drop(['AwayTeam'],axis=1)

In [16]:
df1_final['TotalGoalsScored'] = df1_final['FTHG'] + df1_final['FTAG']
df1_final.rename(columns={'HomeTeam':'Team'}, inplace=True)
df1_final = df1_final.drop(['FTHG','FTAG'],axis=1)
#combine columns and relabel 

In [17]:
df1_final.head()

,Team,TotalGoalsScored
0,Aachen,30
1,Augsburg,36
2,Bayern Munich,77
3,Bochum,41
4,Braunschweig,37


## Total Wins and Losses

Starting with wins...

In [18]:
df1_final['TotalWins'] = np.nan  ##create a blank total wins column

In [19]:
df1_final.head()

,Team,TotalGoalsScored,TotalWins
0,Aachen,30,NaN
1,Augsburg,36,NaN
2,Bayern Munich,77,NaN
3,Bochum,41,NaN
4,Braunschweig,37,NaN


We will find Total Wins by creating a list of winning teams, then count the number of times the teams appear in the list

In [20]:
list_of_winners = []
for x in range(0,len(df_original)):
    if df_original.FTR[x] == 'H':
        list_of_winners.append(df_original.HomeTeam[x])
    if df_original.FTR[x] == 'A':
        list_of_winners.append(df_original.AwayTeam[x])

#making a list of the winners of every game

In [21]:
list = []
for x in df1_final['Team']:
    list.append(list_of_winners.count(x))
    
                #getting an ordered list of wins for each team in the dataframe

In [22]:
df1_final['TotalWins'] = list       ##inserting the values into the dataframe

In [23]:
df1_final.head()

,Team,TotalGoalsScored,TotalWins
0,Aachen,30,6
1,Augsburg,36,8
2,Bayern Munich,77,23
3,Bochum,41,10
4,Braunschweig,37,10


and now for the losses...!

In [24]:
df1_final['TotalLosses'] = np.nan  ##create a blank total losses column

In [25]:
list_of_losers = []
for x in range(0,len(df_original)):
    if df_original.FTR[x] == 'H':
        list_of_losers.append(df_original.AwayTeam[x])
    if df_original.FTR[x] == 'A':
        list_of_losers.append(df_original.HomeTeam[x])

                                #making a list of the losers of every game

In [26]:
list = []
for x in df1_final['Team']:
    list.append(list_of_losers.count(x))
    
                                #getting an ordered list of wins for each team in the dataframe

In [27]:
df1_final['TotalLosses'] = list       ##inserting the values into the dataframe

In [28]:
df1_final.head(10)

,Team,TotalGoalsScored,TotalWins,TotalLosses
0,Aachen,30,6,15
1,Augsburg,36,8,12
2,Bayern Munich,77,23,7
3,Bochum,41,10,17
4,Braunschweig,37,10,9
5,Cottbus,30,8,15
6,Dortmund,80,25,3
7,Dresden,50,12,13
8,Duisburg,42,10,15
9,Ein Frankfurt,76,20,6


## Total Wins in Rain  

Game plan:

Access the Weather Data, using an API

Separate off those games where it rained

Redo steps from 2.1 and 2.2 on the collection of rainy games 

First, we went to the DarkSky API page, and registered, recieving a token which will be the parameter needed to use this API. We saved this token in a .env file, which we will access below:

In [29]:
from dotenv import load_dotenv
load_dotenv() 
## importing the needed library and creating an instance, allowing us to access the text in our .env file

token = os.getenv('DATAKEY')
## getting the specific token needed to use the API

In [30]:
df1['WasItRaining?'] = np.nan 
#creating a blank column we'll use to notate rainy games

In [31]:
df1.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,WasItRaining?
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,NaN
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,NaN
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,NaN
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,NaN
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,NaN


In [32]:
df1['Year'] = df1.apply(lambda _: '', axis=1)
df1['Month'] = df1.apply(lambda _: '', axis=1)
df1['Day'] = df1.apply(lambda _: '', axis=1)

for x in range(len(df1)):
    df1.at[x,'Year']= df1.at[x,'Date'].split('-')[0]
    df1.at[x,'Month'] = df1.at[x,'Date'].split('-')[1]
    df1.at[x,'Day'] = df1.at[x,'Date'].split('-')[2]

#prepare the data to be in a format that is compatibe with the API

In [33]:
df1.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,WasItRaining?,Year,Month,Day
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,NaN,2012,03,31
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,NaN,2011,12,11
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,NaN,2011,08,13
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,NaN,2011,11,27
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,NaN,2012,02,18


Now, lets collect the rain data using our API!

We have simplified this by using the latitude and longitude of Berlin, instead of the latitute and longitude of each specific home team as the locations. We could look to improve this method down line, but for now its just a bit of practice on APIs; not to worry. 

In [34]:
total_weather_data = []
for x in range(len(df1)):
    lat = '52.5200'
    long = '13.4050'
    year= df1['Year'][x]
    month= df1['Month'][x]
    day= df1['Day'][x]
    hours= '15'
    minutes= '00'
    seconds= '00'
    resp = requests.get('https://api.darksky.net/forecast/{}/{},{},{}-{}-{}T{}:{}:{}?units=si&exclude=daily,minutely,hourly,alerts,flags'.format(token,lat,long,year,month,day,hours,minutes,seconds))
    weather_data = resp.json()
    total_weather_data.append(weather_data)
    
#Get the data!

In [35]:
df1['WeatherSummary'] = df1.apply(lambda _: '', axis=1)  ##create a blank column to input useful data

In [37]:
for x in range(len(total_weather_data)):
    ##input the useful data
    if 'precipIntensity' in total_weather_data[x]['currently'].keys():
        if total_weather_data[x]['currently']['precipIntensity'] > 0:
            df1.at[x,'WeatherSummary'] = "rain"
        else:
            df1.at[x,'WeatherSummary'] = "clear"
    else:
        df1.at[x,'WeatherSummary'] = "no data"

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyError: 'currently'

In [39]:
df1.head(10)  ##check the dataframe looks correct

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,WasItRaining?,Year,Month,Day,WeatherSummary
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,NaN,2012,03,31,clear
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,NaN,2011,12,11,clear
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,NaN,2011,08,13,clear
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,NaN,2011,11,27,clear
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,NaN,2012,02,18,clear
5,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,NaN,2012,01,20,clear
6,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,NaN,2012,02,04,clear
7,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,NaN,2012,04,21,clear
8,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,NaN,2011,09,18,clear
9,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,NaN,2011,10,23,rain


In [40]:
df1['WeatherSummary'].value_counts() ##see what values are rainy ones 

clear      362
           224
rain        19
no data      7
Name: WeatherSummary, dtype: int64

In [41]:
df1['WeatherSummary'].isna().sum() 

0

In [42]:
df1.loc[df1['WeatherSummary'].isin(['rain']),'WasItRaining?'] = True
#use the relevant data to decide which matches were rainy

In [43]:
df1['WasItRaining?'].value_counts()

True    19
Name: WasItRaining?, dtype: int64

Now we have the information to make a dataframe with just raining matches!

In [44]:
df1_raining = df1.loc[df1['WasItRaining?'] == True]
#create our rainy matches tables! Now we essentially repeat step 2 to get WinsInRain

In [45]:
df1_raining.drop(['Match_ID','Div','Season','Date','FTHG','FTAG','WasItRaining?','Year','Month','Day','WeatherSummary'], inplace = True,axis=1)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [46]:
df1_raining.reset_index(inplace = True)

In [47]:
df1_raining.drop(columns='index',inplace = True)

In [48]:
df1_raining.head()

,HomeTeam,AwayTeam,FTR
0,Hannover,Bayern Munich,H
1,M'gladbach,Dortmund,D
2,Hansa Rostock,Dresden,D
3,Cottbus,Duisburg,D
4,Ein Frankfurt,Duisburg,H


In [49]:
list_of_winners2 = []
for x in range(len(df1_raining)):
    if df1_raining.FTR[x] == 'H':
        list_of_winners2.append(df1_raining.HomeTeam[x])
    if df1_raining.FTR[x] == 'A':
        list_of_winners2.append(df1_raining.AwayTeam[x])
        
##create a list of winners like before

In [50]:
list2 = []
for x in df1_final['Team']:
    if x in list_of_winners2:
        list2.append(list_of_winners2.count(x))
    else:
        list2.append(0)
##create a list of number of wins of teams in order of how they appear in the final dataframe


In [51]:
df1_final['WinsInRain'] = df1_final.apply(lambda _: '', axis=1)
df1_final['GamesInRain'] = df1_final.apply(lambda _: '', axis=1)

In [52]:
df1_final['WinsInRain'] = list2
#set Wins in Rain to the correct values

In [53]:
df1_final

,Team,TotalGoalsScored,TotalWins,TotalLosses,WinsInRain,GamesInRain
0,Aachen,30,6,15,0,
1,Augsburg,36,8,12,0,
2,Bayern Munich,77,23,7,2,
3,Bochum,41,10,17,0,
4,Braunschweig,37,10,9,0,
5,Cottbus,30,8,15,0,
6,Dortmund,80,25,3,0,
7,Dresden,50,12,13,0,
8,Duisburg,42,10,15,0,
9,Ein Frankfurt,76,20,6,2,


## Total Games in Rain

Our little twist here again, we take ALL team names that played in the rainy matches, not just those that won

In [54]:
list_of_raining_teams = []
for x in range(len(df1_raining)):
        list_of_raining_teams.append(df1_raining.HomeTeam[x])
        list_of_raining_teams.append(df1_raining.AwayTeam[x])

In [55]:
list3 = []
for x in df1_final['Team']:
    if x in list_of_raining_teams:
        list3.append(list_of_raining_teams.count(x))
    else:
        list3.append(0)

In [56]:
df1_final['GamesInRain'] = list3
#as before 

In [57]:
df1_final

,Team,TotalGoalsScored,TotalWins,TotalLosses,WinsInRain,GamesInRain
0,Aachen,30,6,15,0,0
1,Augsburg,36,8,12,0,0
2,Bayern Munich,77,23,7,2,3
3,Bochum,41,10,17,0,1
4,Braunschweig,37,10,9,0,0
5,Cottbus,30,8,15,0,1
6,Dortmund,80,25,3,0,1
7,Dresden,50,12,13,0,1
8,Duisburg,42,10,15,0,2
9,Ein Frankfurt,76,20,6,2,2


## Percentage of Wins in Rain

In [58]:
df1_final['WinPercentageInRain'] = df1_final["WinsInRain"] / df1_final["GamesInRain"] *100

In [59]:
df1_final["WinPercentageInRain"].fillna("No Games in Rain", inplace = True) 

In [60]:
df1_final.drop(columns=['WinsInRain','GamesInRain'],inplace = True)

In [61]:
df1_final.set_index('Team', inplace=True)

# Making the Visulisation  

This function winplot has been written into our libraries file, which we have imported

In [65]:
lib.winplot('Aachen',df1_final)

NameError: name 'df1_final' is not defined

The visulizations for all the teams are easily looked at using this function!

# Exporting Data to a MongoDB file 

Now, we are going to create a MongoDB and upload our newly found data into it.

In [72]:
mongo = lib.MongoHandler() ##initialising an instant using our MongoHandler class we wrote

NameError: name 'pymongo' is not defined

In [73]:
mongo.upload(df1_final, myCollection) ##uploading the dataframe into Mongo!

NameError: name 'mongo' is not defined

In [74]:
[i for i in mycollection.find()] ##having a look to check that it is there!

NameError: name 'mycollection' is not defined

# Conclusion 

We've queried an SQL database, calculated some statistics on the data, created visulations, and stored the data in a Mongo DB for practice with alternative data storing methods! We even cleaned up a bit of the code by refractoring it into our own library that we wrote and imported. Job well done.